# Programming Game AI with Halite

[Halite](https://halite.io/) is a game in which you control a fleet of ships which fly around the galaxy to collect the rare mineral 'halite'. 

![Example halite game](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F3258%2F73a73a0b4a807a7a9e674a40c55f7396%2Fhalite.gif?generation=1594994852379393&alt=media)

The [Halite page on Kaggle](https://www.kaggle.com/c/halite) is the definitive source for the rules of the game and the [Halite SDK](https://www.kaggle.com/sam/halite-sdk-overview) that we'll use to program bots to play the game.

First, set up an environment to test our Halite bots:

In [ ]:
# Set Up Environment
from kaggle_environments import evaluate, make
env = make("halite", configuration={ "episodeSteps": 400 }, debug=True)
print (env.configuration)

# Imports helper functions
from kaggle_environments.envs.halite.helpers import *

Now we'll create the world's worst bot: each ships takes no action at all. This is achieved by setting the ship's action to `None`.

In [ ]:
%%writefile nullbot.py
from kaggle_environments.envs.halite.helpers import *
# Returns the commands we send to our ships and shipyards
def agent(obs, config):
    board = Board(obs, config)
    me = board.current_player
    
    for ship in me.ships:
        ship.next_action = None
                
    return me.next_actions

Let's see that bot in action against a bot that takes completely random moves.

In [ ]:
env = make("halite", debug=True)
env.run(["nullbot.py", "random"])
env.render(mode="ipython", width=800, height=600)

Here is a slightly better bot: the ships always move south, unless there is more than 100 halite in the current cell.

In [ ]:
%%writefile southbot.py
from kaggle_environments.envs.halite.helpers import *
# Returns the commands we send to our ships and shipyards
def agent(obs, config):
    size = config.size
    board = Board(obs, config)
    me = board.current_player
    
    for ship in me.ships:
        if ship.cell.halite > 100:
            ship.next_action = None
        else:
            ship.next_action = ShipAction.SOUTH
                
    return me.next_actions

Let's run two of these bots against each other:

In [ ]:
env = make("halite", debug=True)
env.run(["southbot.py", "southbot.py"])
env.render(mode="ipython", width=800, height=600)

Now let's create a shipyard, which will be used to spawn ships every second turn:

In [ ]:
%%writefile spawnbot.py
from kaggle_environments.envs.halite.helpers import *
from random import choice

# Returns the commands we send to our ships and shipyards
def agent(obs, config):
    
    size = config.size
    board = Board(obs, config)
    me = board.current_player
    directions = [ShipAction.NORTH, ShipAction.SOUTH, ShipAction.EAST, ShipAction.WEST]
    
    if len(me.shipyards) == 0 and len(me.ships) > 0:
        me.ships[0].next_action = ShipAction.CONVERT
    
    if board.step % 2 == 0 and len(me.shipyards) > 0 and me.shipyards[0].cell.ship == None:
        me.shipyards[0].next_action = ShipyardAction.SPAWN

    for ship in me.ships:
        if ship.next_action == None:
            if ship.cell.halite > 100:
                ship.next_action = None
            elif ship.cell.south.ship == None:
                ship.next_action = ShipAction.SOUTH
                
    return me.next_actions

In [ ]:
env = make("halite", debug=True)
env.run(["spawnbot.py", "spawnbot.py"])
env.render(mode="ipython", width=800, height=600)